In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install qwen-vl-utils

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-51k4j9kr
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-51k4j9kr
  Resolved https://github.com/huggingface/transformers to commit bebeccb06a3c10cc8fedf27faaa8ed904df3353b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 9.9 MB/s eta 0:00:00ta 0:00:01
  Created wheel for transformers: filename=transformers-4.56.0.dev0-py3-none-any.whl size=12496743 sha256=891b974d7ec5a26992fcebe3f7861b2a146e041f731d257203dacc0b96b2c1de
  Stored in directory: /tmp/pip-ephem-wheel-cache-0bkj0qk_/wheels/04/a3/f1/b88775f8e1665827525b19ac7590250f1038d947067beba9fb
Successfully built transformers
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.33.1
    Uninstalling huggingface-hub

In [2]:
# From Hugging Face Transformers – used for multimodal document/image-text understanding
# Utility function, likely for processing visual-language inputs (e.g., OCR, layout)
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

2025-08-19 15:19:51.761834: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755616792.128497      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755616792.230563      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
# Load the model on the available device(s)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto"
)

# default processer
min_pixels = 256 * 28 * 28
max_pixels = 1024 * 28 * 28
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", 
                                          min_pixels=min_pixels, 
                                          max_pixels=max_pixels)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "/kaggle/input/kan-test/kanShree.jfif",
            },
            {"type": "text", "text": (
                "You are an expert form extraction AI."
                "Your task is to analyze the provided image, which is an application form for an Annual Scholarship"
                "Extract all including text entries, numerical values, and selected options from checkboxes/radio buttons."
                "**Instructions:**"
                "1.  **Output Format:** Provide the extracted data as a single JSON object."
                "2.  **Field Names:**"
                "YEAR"
                "Form No."
                "Application ID"
                "School/Institution Name"
                "DISE Code/Equivalent Code"
                "Block/Municipality"
                "District"
                "Applicant's Name"
                "DOB (dd/mm/yyyy)"
                "Class"
                "Father's Name"
                "Mother's Name"
                "EPIC No. of Father/Mother"
                "Legal Guardian's (if any) Name"
                "Applicant's Marital Status"
                "Percentage of disability as per Disability Certificate"
                "Caste"
                "Religion"
                "AADHAAR Card No."
                "Signature of the Aadhaar Number Holder/ I Agree"
                "Name of the applicant"
            )},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

['```json\n{\n  "Caste": "General",\n  "Religion": "Hindu",\n  "AADHAAR Card No": "7306 5813 9934",\n  "Signature of the Aadhaar Number Holder/ I Agree": "Dipsikha Bera",\n  "Name of the applicant": "DIPSIKHA BERA"\n}\n```']
